# Decorators

I could probably write a small book about **object-oriented programming** (referred to as **OOP** henceforth) and classes. In this chapter, I'm facing the hard challenge of finding the balance between breadth and depth. There are simply too many things to tell, and there's plenty of them that would take more than this whole chapter if I described them alone in depth. Therefore, I will try to give you what I think is a good panoramic view of the fundamentals, plus a few things that may come in handy in the next chapters. Python's official documentation will help in filling the gaps.

We're going to explore three important concepts in this chapter: decorators, OOP, and iterators

# Decorators
In the previous chapter, I measured the execution time of various expressions. If you recall, I had to initialize a variable to the start time, and subtract it from the current time after execution in order to calculate the elapsed time. I also printed it on the console after each measurement. That was very tedious

Every time you find yourself repeating things, an alarm bell should go off. Can you out that code in a function and avoid repitition? The answer most of the time is yes, so let's look at an example.

#### decorators/time.measure.start.py

In [19]:
from time import sleep, time

def f():
    sleep(.3)

def g():
    sleep(.5)
    

t = time()
f()
print('f took:', time() -t)

f took: 0.3098311424255371


In [20]:
t = time()
g()
print('g took:', time() -t)

g took: 0.5066518783569336


In the preceding code, I defined two functions, **f** and **g**, which do nothing but sleep (by 0.3 and 0.5 seconds respectively). I used the **sleep** function to suspend the execution for the desired amount of time. I also highlighted how we calculate the time elapsed by setting **t** to the current time and then subracting it when the task is done. You can see that the measure is pretty accurate.

Now, how do we avoid repeating that code and those calculations? One first potential approach could be the following:

#### decorators/time.measure.dry.py

In [21]:
from time import sleep, time

def f():
    sleep(.3)
    
def g():
    sleep(.5)
    

def measure(func):
    t = time()
    func()
    print(func.__name__, 'took:', time() -t)
    
    
measure(f)
    

f took: 0.30698633193969727


In [22]:
measure(g)

g took: 0.5025012493133545


Ah, much better now. The whole timing mechanism has been encapsulated into a function so we don't repeat code. We print the function name dynamically and it's easy to enough to code. What if we need to pass arguments to the function we measure? This code would get just a bit more complicated, so let's see an example.

#### decorators/time.measure.arguments.py

In [23]:
from time import sleep, time

def f(sleep_time=0.1):
    sleep(sleep_time)
    
def measure(func, *args, **kwargs):
    t = time()
    func(*args, **kwargs)
    print(func.__name__, 'took:', time() -t)
    
measure(f, sleep_time=0.3)

f took: 0.30245399475097656


In [24]:
measure(f, 0.2)

f took: 0.20849299430847168


In [ ]:
measure(f)

Now, **f** is expecting to be fed **sleep_time** (with a default value of 0.1). I also had to change the **measure** function so that it is now accepting a function, any variable positional arugments, and any variable keyword arguments. In this way, whatever we call **measure** with, we redirect those arguments to the call to **f** we do inside.

This is very good, but we can push it a little bit further. Let's say we want to somehow have that timing behaviour built-in in the **f** function, so that we could just call it and have that measure taken. Here's how we caould do it:

#### decorators/time.measure.deco1.py

In [16]:
from time import sleep, time

def f(sleep_time=0.1):
    sleep(sleep_time)
    
def measure(func):
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() -t)
    return wrapper

f = measure(f)

In [17]:
f(0.2)

f took: 0.20227313041687012


The preceding code is probably not so straightforward. I confess that, even today, it sometimes requires me some serious concentration to understand some decorators, they can be pretty nasty. Let's see what happens here. The magic is in the **decoration point**. We basically reassign **f** with whatever is returned by **measure** when we call it with **f** as an argument. Within **measure**., we define another function, **wrapper**, and then we return it. So, the net effect is that after the decoration point, when we call **f**, we're actually calling **wrapper**. Since the wrapper inside is calling **func**, which is **f**, we are actually closing the loop like that. If you don't believe me, take a look at the last line.

**wrapper** is actually... a wrapper. It takes varaible and positional arguments, and calls **f** with them. It also does the time measurement trick around the call.

This technique is called **decoration**, and **measure** is, at all effects, a **decorator**. This paradigm becamse so popular and widely used that at some point, Python added a special syntax for it (check **PEP 318**). Let's explore three cases: one decorator, two decorators, and one decorator that takes arguments.

#### decorators/syntax.py

In [18]:
def func(arg1, arg2, ...):
    pass
func = decorator(func)

# is equivalent to the following:
@decorator
def func(arg1, arg2, ...):
    pass



SyntaxError: invalid syntax (<ipython-input-18-949dd832708d>, line 1)

Basically, instead of manually reassigning the function to what was returned by the decorator, we prepend the definition of the function with the special syntax
***@decorator_name***.

We can apply multiple decorators to the same function in the following way:

##### decorators/syntax.py

In [ ]:
def func(arg1, arg2, ...):
    pass
func = deco1(deco2(func))

# is equivalent to the following:

@deco1
@deco2
def func(arg1, arg2, ...):
    pass



When applying multiple decorators, pay attention to the order, should it matter. In the preceding example, **func** is decorated with **deco2** first, and the result is decorated with **deco1**. A good rule of thumb is: *the closer the decorator to the function, the sooner it is applied*.

Some decorators can take arguments. This technique is generally used to produce other decorators. Let's look at the syntax, and then we'll see an example of it.

##### decorators/syntax.py

In [ ]:
def func(arg1, arg2, ...):
    pass

func decoarg(argA, argB)(func)

# is equivalent to the following:

@decoarg(argA, argB)
def func(arg1, arg2, ...):
    pass

As you can see, this case is a bit different. First **decoarg** is called with the given arguments, and then its return value (the actual decorator) is called with **func**. Before I give you another example, let's fix one thing that is bothering me. I don't want to lose the original function name and docstring (and the other attributes as well, check the documentation for the details) when I decorate it. But because inside our decorator we return **wrapper**, the original attributes from **func** are lost and **f** ends up being assigned the attributes of **wrapper**. There is an easy fix for that from **functools**, a wonderful module from the Python standard library. I will fix the last example, and I will also rewrite its syntax to use the **@** operator.

###### decorators/time.measure.deco2.py

In [ ]:
from time import sleep, time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        func(*args, **kwargs)
        print(func.__name__, 'took:', time() - t)
    return wrapper

@measure
def f(sleep_time=0.1):
    """I'm a cat. I love to sleep!"""
    sleep(sleep_time)
    
f(sleep_time=0.3)

In [ ]:
print(f.__name__, ':', f.__doc__)

Now we're talking! As you can see, all we need to do is to tell Python that **wrapper** actually wraps **func** (by means of the **wraps** function), and you can see that the original name and docstring are now maintained.

Let's see another example. I want a decorator that prints an error message when the result of a function is greater than a threshold. I will also take this opportunity to show you how to apply two decorators at once.

###### decorators/two.decorators.py

In [ ]:
from time import sleep, time
from functools import wraps

def measure(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time()
        result = func(*args, **kwargs)
        print(func.__name__, 'took:', time() -t)
        return result
    return wrapper

def max_result(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if result > 100:
            print(
                'Result is too big ({0}). Max allowed is 100.'.format(result))
        return result
    return wrapper

@measure
@max_result
def cube(n):
    return n**3

print(cube(2))


In [ ]:
print(cube(5))

# A decorator factory

Let's simplify this example now, going back to a single decorator: **max_result**. I want to make it so that I can decorate different functions with different thresholds, and I don't want to write one decorator for each threshold. Let's amend **max_result** so that it allows us to decorate functions specifying the threshold dynamically.

###### decorators/decorators.factory.py

In [ ]:
def max_result(threshold):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            if result > threshold:
                print(
                    'Result is too big ({0}). Max allowed is {1}.'.format(result, threshold))
            return result
        return wrapper
    return decorator

@max_result(75)
def cube(n):
    return n ** 3

print(cube(5))

This preceding code shows you how to write a **decorator factory**. If you recall, decorating a function with a decorator that takes arguments is the same as writing **func = decorator(argA, argB)(func)**, so when we decorate **cube** with **max_resul(75)**, we're doing **cube = max_result(75)(cube)**.

Let's go through what happens, step by step. When we call **max_result(75)**, we enter its body. A **decorator** function is defined inside, which takes a function as its only argument. Inside that function, the usual decorator trick is performed. We define a wrapper, inside of which we check the result of the original function's call. The beauty of this approach is that from the innermost level, we can still refer to both **func** and **threshold**, which allows us to set the threshold dynamically.

**wrapper** returns **result**, **decorator** returns **wrapper**, and **max_result** returns **decorator**. This means that our call **cube = max_result(75)(cube)**, actually becomes **cube = decorator(cube)**. Not just any **decorator** though, but one for which **threshold** has the value **75**. This is achieved by a mechanism called **closure**, which is outside of the socre of this chapter but nonetheless very interesting, so I mentioned it for you to do some research on it.

Running the last examples produces the following result:

In [ ]:
@max_result(75)
def cube(n):
    return n ** 3

@max_result(100)
def square(n):
    return n ** 2

@max_result(1000)
def multiply(a, b):
    return a * b

Note that every decoration uses different **threshold** value.

Decorators are very popular in Python. They are used quite often and they simplify (and beautify, I dare say) the code a lot.

# The Power of Decorators
At their core, Python's decorators allow you to extend and modify the behavior of a callable (functions, methods, and classes) *without* permanently modifying the callable itself.

Any sufficiently generic functionality you can tack on to an existing class or function's behavior makes a great use case for decoration. This includes the following:

* logging
* enforcing access control and authentication
* instrumentation and timing functions
* rate-limiting
* caching, and more

Now, why should you master the use of decorators in Python? After all, what I just mentioned sounded quite abstract, and it might be difficult to see how decorators can benefit you in your day-to-day work as a Python developer. Let me try to bring some clarity to this question by giving you a somewhat real-world example:

Imagine you've got 30 functions withs business logic in yhour report-generating program. One rainy Monday your boss walks up to your desk and says: *Happy Monday! Remember those TPS reports? I need you to add input/output logging to each step in the report generator. XYZ Corp needs it for auditing purposes. Oh, and I told them we can ship this by Wednesday."*

Depending on whether or not you've got a solid grasp on Python's decorators, this request will either send your blood pressure spking or leave you relatively calm.

Without decorators you might be spending the next three days scrambling to modify each of those 30 functions and clutter them up with manual logging calls. Fun times, right?

If you do know your decorators however, you'll calmly smile at your boss and say: *"Don't worry Jim, I'll get i done by 2pm today."*

Right after that you'll type the code for a generic @audit_log decorator (that's only 10 lines long) and quickly paste it in front of each function definition. Then you'll commit your code and grab another cup of coffee...

I'm dramatizing here, but only a little. Decorators *can be* that powerful. I'd go as far as to say that understanding decorators is a milestone for any serious Python programmer. They require a solid graps of several advanced concepts in the language, including the properties of *first-class functions.*


**I believe that the payoff for understanding how decorators work in Python can be enormous.**

Sure, decorators are relatively complicated to wrap your head around for the first time, but they're a highly useful feature that you'll often encounter in third-party frameworks and the Python standard library. Explaining decorators is also a *make or break* moment for any good Python tutorial. I'll do my best here to introduce you to them step by step.

Before you dive in however, now would be an excellent moment to refresh your memory on the properties of *first-class functions* in Python. There's a **chapter on them in this book**, and I would encourage you totake a few minutes to review it. The most important "first class functions" takeaways for understanding decorators are:

* **Functions are objects**––they can be assigned to variables and passed to and returned from other functions

* **Functions can be defined inside other functions**––and a child function can capture the parent function's local state (lexical closures)

Alright, are you ready to do this? Let's get started.

## Python Decorator Basics
Now, what are decorators really? They "decorate" or "wrap" another function and let you execute code before and after the wrapped functions runs.

Decorators allow you to define reusable building blocks that can change or extend the behavior of other functions. And, they let you do that without permanently modfying the wrapped function itself. The function's behaviour changes only when it's *decorated*.

What might the implementation of a simple decorator look like? In basic terms, a decorator is a *callable that takes a callable as input and returns another callable.*

The follwoing function has that property and could be considered the simplest decorator you could possibly write:


In [ ]:
def null_decorator(func):
    return func

As you can see, null_decorator is a callable (It's a function), it takes another callable as its input, and it returns the same input callable without modifying it.

Let's use it to *decorate* (or *wrap*) another function:

In [ ]:
def greet():
    return 'Hello'

greet = null_decorator(greet)
greet()


In this example, I've defined a greet function and then immediately decorated it by running it through the null_decorator function. I know this doesn't look very useful yet. I mean, we specifically designed the null decorator to be useless, right? But in a moment this example will clarify how Python's special-case decorator syntax works.

Instead of explicitly calling null_decortor on greet and then reassigning the greet variable, you can use Python @ syntax for decorating a function more conveniently:

In [ ]:
@null_decorator
def greet():
    return 'Hello!'

greet()

Putting an @null_decorator line in front of the function definition is the same as defining the function first and then running through the decorator. Using the @ syntax is just *syntactic sugar* and a shortcut for this commonly used pattern.

Note that using the @ syntax decorates the function immediately at definition time. This makes it difficult to access the undecorated original without brittle hacks. Therefore you might choose to decorate some functions manually in order to retain the ability to call the undecorated function as well.

## Decorators Can Modify Behavior

Now that you're a little more familiar with the decorator syntax, let's write another decorator that *actually does something* and modifies the behavior of the decorated function.

Here's a slighlt more complex decorator which converts the result of the decorated function to uppercase letters:

In [ ]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper


Instead of simply returning the input function like the null decorator did, this uppercase decorator defines a new function on the fly (a closure) and uses it to *wrap* the input function in order to modify its behavior at call time.

The wrapper closure has access to the undecorated input function and it is free to execute additional code before and after calling the input function. (Technically, it doesn't even need to call the input function at all.)

Note how, up until now, the decorated function has never been executed. Actually calling the input funcgtion at this point wouldn't make any sense––you'll want the decorator to be able to modify the behavior of its input function when it eventually gets called.

You might want to let that sink in for a minute or two. I know how complicated this stuff can seem, but we'll get it sorted out together, I promise.

Time to see the uppercase decorator in action. What happens if you decorate the original greet function with it?


In [ ]:
@uppercase
def greet():
    return 'Hello!'

I hope this was the result you expected. Let's take a closer look at what just happened here. Unlike null_decorator, our uppercase decorator returns *different function object* when it decorates a function:

In [ ]:
greet

In [ ]:
null_decorator(greet)

In [ ]:
uppercase(greet)


And as you saw earlier, it need to do that in order to modify the behavior of the decorated function when it finally gets called. The uppercase decorator is a function itself. And the only way to influence the "future behavior" of an input function it decorates is to replace(or *wrap*) the input function with a closure.

That's why uppercase defines and returns another function (the closure) that can then be called at a later time, run the original input function, and modify its result.

Decorators modify the behavior of a callable through a wrapper closure so you don't have to permanently modify the original. The original callable isn't permanently modified––its behavior changes only when decorated. 

This lets you tack on reusable building blocks, like logging and other instrumentation, to existing functions and clases. It makes decorators such a powerful feature in Python that it's frequently used in the standard library and in the third-party packages.

# A Quick Intermission

By the way, if you feel like you need a quick coffee break or a walk around the block at this point––that's totally normal. In my opinion closures and decorators are some of the most difficult concepts to understand in Python.

Please, take your time and don't worry about figuring and don't worry about figuring this out immediately. Playing through the code examples in an interpreter session one by one often helps make things sink in.

I know you can do it!

## Applying Multiple Decorators to a Function

Perhaps not surprisingly, you can apply more than one decorator to a function. This accumulates their effects and it's what makes decorators so helpful as reusable building blocks.

Here's an example. The following two decorators wrap the output string of the decorated function in HTML tags. By looking at hos the tags are nested, you can see which order Python uses to apply multiple decorators:

In [ ]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper


Now let's take these two decorators and apply them to our greet function at the same time. You can use the regular @ syntax for that and just "stack" multiple decorators on top of a single function:

In [ ]:
@strong
@emphasis
def greet():
    return 'Hello!'


What output do you expect to see if you run the decorated function? Will the @emphasis decorator add its <em> tag first, or does @strong have precedence? Here's what happens when you call the decorated function:

In [ ]:
greet()

This clearly shows in what order the decorators were applied: from *bottom to top*. First, the input function was wrapped by the @emphasis decorator, and then the resulting (decorated) function got wrapped again by the @strong decorator.

To help me remember this bottom to top order, I like to call this behavior *decorator stacking*. You start building the stack at the bottom and then keep adding new blocks on top to work your way upwards.

If you break down the above example and avoid the @ syntax to apply the decorators, the chain of decorator function calls look like this:

In [ ]:
decorated_greet = strong(emphasis(greet))

Again, you can see that the emphasis decorator is applied first and then the resulting wrapped function is wrapped again by the strong decorator.

This also means that deep levels of decorator stacking will eventually have an effecgt on performance because they keep adding nested function calls. In practice, this usually won't be a problem, but it's something to keep in mind if you're working on performance-intensive code that frequently uses decoration.

## Decorating Functions That Accept Arguments

In [ ]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

There are two notable things going on with this decorator:

* It uses * and ** operator in the wrapper closure definition to collect all positional and keyword arguments and stores them in variables(args and kwargs).

* The wrapper closure then forwards the collected arguments to the original input function using the * and ** "argument unpacking" operators.

It's a bit unfortunate that the meaning of the star and double-star operators is overloaded and changes depending on the context they're used in, but I hope you get the idea.

Let's expand the technique laid out by the proxy decorator into a more useful practical example. Here's a trace decorator that logs function arguments and results during execution time:

In [ ]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(
            f'Trace: calling {func.__name__}() ' 
            f'with {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        print(
            f'TRACE: {func.__name__}() '
            f'returned {original_result!r}')
        return original_result
    return wrapper

Decorating a function with trace and then calling it will print the arguments passed to the decorated function and its return value. This is still somewhat of a "toy" example––but in a pinch it makes a great debugging aid:

In [ ]:
@trace
def say(name, line):
    return f'{name}: {line}'

In [ ]:
say('Jane', 'Hello, World')

Speaking of debugging, there are some things you should keep in mind when debugging decorators:

## How to Write "Debuggable" Decorators

When you use a decorator, really what you're doing is replacing one function with another. One downside of this process is that it "hides" some of the metadataattached to the original (undecorated) function.

For example, the original function name, its docstring, and parameter list are hidden by the wrapper closure:

In [ ]:
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

decorated_greet = uppercase(greet)


In [ ]:
greet.__name__

In [ ]:
greet.__doc__

In [ ]:
decorated_greet.__name__

In [ ]:
decorated_greet.__doc__

This makes debugging and working with the Python interpreter awkward and challenging. Thankfully there's a quick fix for this: the functools.wraps decorator included in Python's standard library.

You can use functools.wraps in your own decorators to copy over the lost metadata from the undecorated function to the decorator closure. Here's an example:

In [ ]:
def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

Applying functools.wraps to the wrapper closure returned by the decorator carries over the docstring and other metadata of the input function:

In [ ]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

In [ ]:
greet.__name__


In [ ]:
greet.__doc__

As a best practice, I'd recommend that you use functools.wraps in all of the decorators you write your self. It doesn't take much time and it will save you (and others) debugging headaches down the road.

Oh, and congratulations––you've made it all the way to the end of this complicated chapter and learned a whole lot about decorators in Python. Great Job!

### Key Takeaways

* Decorators define reusable building blocks you can apply to a callable to modify its behavior without permanently modifying the callable itself.

* The @ syntax is just a shorthand for calling the decorator on an input function. Multiple decorators on a single function are applied bottom to top (*decorator stacking*).

* As a debugging best practices, use the functools.wraps helper in your own decorators to carry over metadata from the undercorated callable to the decorated one.

* Just like any other tool in the software development toolbox decorators are not a cure all and they should not be overused. It's important to blanace the need to "get stuff done" with the goal of "not getting tangled up in a horrible, unmaintainable mess of code base." 